In [12]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# --------------------------------------------
# ----------------- Mock Data ----------------
# --------------------------------------------

games = ["GTA V", "Zelda", "Minecraft", "Fortnite", "Skyrim"]
player_counts = [200000, 180000, 160000, 120000, 100000]
years = list(range(1970, 2025))

df = pd.DataFrame({
    "Game": games * 10,
    "Player Count": [val + (i * 5000 % 15000) for i, val in enumerate(player_counts * 10)]
})

# --------------------------------------------
# -------------- Dash App Setup --------------
# --------------------------------------------

app = Dash(__name__)
app.title = "Dashboard"

# --------------------------------------------
# ---------------- Tabs Config ---------------
# --------------------------------------------

tabs = [
    {'label': 'Most Popular', 'value': 'most-popular'},
    {'label': 'Companies', 'value': 'companies'},
    {'label': 'New Releases / Coming Soon', 'value': 'new-releases'},
    {'label': '...', 'value': 'placeholder'}
]

# --------------------------------------------
# ---------- Most Popular Tab Layout ---------
# --------------------------------------------

def most_popular_layout():
    return html.Div([
        # Top Filter Bar
        html.Div([
            html.Div("BY:", style={'padding': '0 10px', 'fontWeight': 'bold'}),
            *[html.Button(label, style={'margin': '0 5px'}) for label in
              ["Twitch", "YouTube", "Reddit", "Addings", "Rating", "Metacritic", "Games", "Genres"]]
        ], style={
            'display': 'flex', 'padding': '10px',
            'backgroundColor': '#a3d2ff', 'borderBottom': '1px solid #ccc'
        }),

        # Main Layout: Boxplot + Right Sidebar
        html.Div([
            # Center Boxplot
            html.Div([
                dcc.Graph(
                        id='barchart',
                        figure=px.bar(df.groupby("Game", as_index=False)["Player Count"].mean(),
                          x="Game", y="Player Count", title="Player Count per Game")
                        )
            ], style={'flex': 1, 'padding': '10px'}),

            # Right Sidebar
            html.Div([
                dcc.Input(
                    id="search",
                    type="text",
                    placeholder="Search games...",
                    style={'width': '90%', 'margin': '10px'}
                ),
                html.Div([
                    html.Div(f"{i+1}. {game}", style={
                        'padding': '5px', 'borderBottom': '1px solid #ccc'
                    }) for i, game in enumerate(games)
                ], style={
                    'overflowY': 'scroll', 'height': '500px', 'padding': '0 10px'
                })
            ], style={
                'width': '200px', 'backgroundColor': '#a3d2ff', 'paddingTop': '10px'
            })
        ], style={'display': 'flex', 'flexDirection': 'row', 'height': '600px'}),

        # Bottom Year Scroller
        html.Div([
            html.Div([
                html.Button(str(year), style={'margin': '5px'}) for year in years
            ], style={
                'display': 'flex', 'overflowX': 'scroll', 'padding': '10px 0'
            })
        ], style={
            'backgroundColor': '#a3d2ff', 'borderTop': '1px solid #ccc'
        })
    ])

# --------------------------------------------
# ----------- Companies Tab Layout -----------
# --------------------------------------------

def companies_layout():
    # Mock data
    company_name = "Nintendo"
    company_data = pd.DataFrame({
        "Game": ["Mario", "Zelda", "Animal Crossing", "Metroid", "Pikmin"],
        "Units Sold (M)": [40, 35, 25, 15, 10]
    })

    return html.Div([
        # Top bar
        html.Div([
            html.Div(f"Company: {company_name}", style={
                'fontSize': '20px',
                'fontWeight': 'bold',
                'padding': '10px'
            }),
            dcc.Input(
                id="company-search",
                type="text",
                placeholder="Search company...",
                style={'marginLeft': 'auto', 'marginRight': '10px', 'padding': '5px'}
            )
        ], style={
            'display': 'flex',
            'alignItems': 'center',
            'justifyContent': 'space-between',
            'backgroundColor': '#a3d2ff',
            'padding': '10px 20px',
            'marginBottom': '20px'
        }),

        # Bar chart
        html.Div([
            dcc.Graph(
                id='company-bar',
                figure=px.bar(
                    company_data,
                    x="Game",
                    y="Units Sold (M)",
                    title=f"{company_name} - Top Game Sales",
                    text="Units Sold (M)"
                )
            )
        ], style={'padding': '0 20px'})
    ])
    
# --------------------------------------------
# ---------------- Main Layout ---------------
# --------------------------------------------

app.layout = html.Div([
    html.Div([
        html.H1("Dashboard", style={'margin': '10px 20px'}),
        dcc.Tabs(
            id="main-tabs",
            value='most-popular',
            children=[dcc.Tab(label=tab['label'], value=tab['value']) for tab in tabs],
            style={'margin': '0 20px'}
        )
    ], style={'backgroundColor': '#f0f0f0', 'paddingBottom': '10px'}),

    html.Div(id='tab-content', style={'padding': '20px'})
])

# --------------------------------------------
# ------ Callback to Switch Tab Content ------
# --------------------------------------------

@app.callback(
    Output('tab-content', 'children'),
    Input('main-tabs', 'value')
)
def render_content(tab):
    if tab == 'most-popular':
        return most_popular_layout()
    elif tab == 'companies':
        return companies_layout()
    elif tab == 'new-releases':
        return html.Div([html.H3("New Releases / Coming Soon")])
    elif tab == 'placeholder':
        return html.Div([html.H3("Future Tabs")])
    return html.Div()

# --------------------------------------------
# -------------------- Run -------------------
# --------------------------------------------
if __name__ == '__main__':
    app.run(debug=True)
